## Analysis of Models Home Credit - Credit Risk Model Stability

https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability

Contents:
- [Importing Data and Pre-Processing](#load_data)

In [2]:
# import packages

import os
from pyprojroot import here
# set working directory to root of github project for consistent access to helper functions
os.chdir(here())

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import scipy.stats as st
import math
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    KFold,
)

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression

import lightgbm as lgb
import shap

from elements.utils.data_processors import (
    snake_case
)
from elements.utils.model_evaluation import (
    classification_model_report
)

pd.set_option('display.float_format', '{:.4f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Importing Data and Pre-Processing
<a id='load_data'></a>